# DSA-110 Bandpass Calibration (Notebook Test)

This notebook:
- Runs fast QA to recommend a reference antenna.
- Auto-selects the bandpass fields using the VLA calibrator catalog (PB-weighted).
- Executes K/BA/BP/G solves using CASA6 tasks via the pipeline helpers.

Notes:
- Run this with a CASA 6 kernel (casatools/casatasks available).
- If you don’t have CASA in the kernel, run the QA + selection cells, then run the CLI externally.

In [1]:
# Setup: add repo src to the path and define inputs
import os, sys, json
# Be conservative with threading to avoid BLAS/OMP conflicts in CASA
os.environ.setdefault('OMP_NUM_THREADS', '1')
os.environ.setdefault('OPENBLAS_NUM_THREADS', '1')
os.environ.setdefault('MKL_NUM_THREADS', '1')
os.environ.setdefault('NUMEXPR_NUM_THREADS', '1')
sys.path.insert(0, os.path.abspath('../../src'))  # ensure dsa110_contimg is importable

# Measurement Set path (adjust as needed)
MS = '/scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms'

# VLA calibrator catalog
CAL_CATALOG = '/data/dsa110-contimg/data-samples/catalogs/vlacalibrators.txt'

# Auto-selection configuration
BP_WINDOW = 3                 # number of fields around the peak (approx)
CAL_SEARCH_RADIUS_DEG = 2.0  # search radius for catalog candidates (deg)

# QA output directory
QA_DIR = '/tmp/qa-auto'
os.makedirs(QA_DIR, exist_ok=True)

MS

'/scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms'

In [2]:
# 1) Run fast QA to rank reference antennas
from dsa110_contimg.qa.fast_plots import run_fast_plots

artifacts = run_fast_plots(MS, output_dir=QA_DIR)
print('Fast QA artifacts:', artifacts)

rec_path = os.path.join(QA_DIR, 'refant_ranking.json')
with open(rec_path, 'r', encoding='utf-8') as f:
    ranking = json.load(f)
refant = ranking.get('recommended', {}).get('antenna_id')
print('Recommended refant:', refant)
assert refant is not None, 'No recommended reference antenna found'
refant

Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms: 25 columns, 111744 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms::DATA_DESCRIPTION: 3 columns, 1 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms::SPECTRAL_WINDOW: 14 columns, 1 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms::ANTENNA: 8 columns, 117 rows
Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms::ANTENNA: 8 columns, 117 rows
Fast QA artifacts: ['amp_vs_time_fast.png', 'amp_vs_freq_fast.png', 'phase_vs_freq_fast.png', 'uv_plane_fast.png', 'per_antenna_metrics_fast.png', 'phase_sigma_sorted_fast.png', 'coherence_vs_flag

67

In [3]:
# 2) Auto-select bandpass fields from the VLA calibrator catalog
from dsa110_contimg.calibration.selection import select_bandpass_from_catalog

sel_str, indices, wflux, calinfo = select_bandpass_from_catalog(
    MS, CAL_CATALOG, search_radius_deg=CAL_SEARCH_RADIUS_DEG, window=BP_WINDOW
)
name, ra_deg, dec_deg, flux_jy = calinfo
print(f'Calibrator: {name} (RA {ra_deg:.4f} deg, Dec {dec_deg:.4f} deg, flux {flux_jy:.3f} Jy)')
print('Selected fields:', sel_str, 'indices:', indices)

# Optional: quick plot of PB-weighted flux per field
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(8,3))
plt.plot(np.arange(len(wflux)), wflux, '-o', ms=3)
plt.axvspan(indices[0]-0.5, indices[-1]+0.5, color='orange', alpha=0.2, label='selected')
plt.xlabel('Field index')
plt.ylabel('PB-weighted flux (Jy)')
plt.title('Bandpass field auto-selection')
plt.legend()
plt.tight_layout()
plt.show()

sel_str

Successful readonly open of default-locked table /scratch/dsa110-contimg/data-samples/ms/0834_555_transit/2025-10-03T15:15:58.ms::FIELD: 9 columns, 24 rows
Calibrator: 0824+558 (RA 126.1968 deg, Dec 55.8785 deg, flux 1.200 Jy)
Selected fields: 22~23 indices: [22, 23]


/tmp/ipykernel_3612793/1060446652.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


'22~23'

In [4]:
# 3) Run K/BA/BP/G solves using CASA (requires casatools/casatasks)
from importlib.util import find_spec

if find_spec('casatasks') is None or find_spec('casatools') is None:
    raise RuntimeError('CASA not available in this kernel. Switch to a CASA 6 kernel.')

from dsa110_contimg.calibration.cli import run_calibrator

print(f'Running calibration...')
print(' MS:', MS)
print(' Fields:', sel_str)
print(' Refant:', refant)
# The calibrate helper patches FIELD::NUM_POLY if needed and skips flagging (do_flagging=False)
tables = run_calibrator(MS, sel_str, str(refant), do_flagging=False)
print('Calibration tables produced:')
for t in tables:
    print(' -', t)
tables

: 